In [1]:
# package imports
import base64
from dotenv import load_dotenv
import json
import lyricsgenius as lg
import math
import numpy as np
import os
import pandas as pd
import requests
import urllib

# relative imports
import auth.spotify as spotify

In [2]:
# read environment variables
load_dotenv()
# client id and secret
SPOTIFY_API_CLIENT_ID = os.getenv('SPOTIFY_API_CLIENT_ID')
SPOTIFY_API_CLIENT_SECRET = os.getenv('SPOTIFY_API_CLIENT_SECRET')
# refresh token
SPOTIFY_REFRESH_TOKEN = os.getenv('SPOTIFY_REFRESH_TOKEN')
# api urls
SPOTIFY_API_BASE_URL = os.getenv('SPOTIFY_API_BASE_URL')
SPOTIFY_API_TOKEN_URL = os.getenv('SPOTIFY_API_TOKEN_URL')
# redirect uri
SPOTIFY_API_REDIRECT_URI = os.getenv('SPOTIFY_API_REDIRECT_URI')
# api paths
ALBUMS_ENDPOINT = os.getenv('ALBUMS_ENDPOINT')
ARTISTS_ENDPOINT = os.getenv('ARTISTS_ENDPOINT')
TRACKS_ENDPOINT = os.getenv('TRACKS_ENDPOINT')
TRACK_AUDIO_FEATURES_ENDPOINT = os.getenv('TRACK_AUDIO_FEATURES_ENDPOINT')
USER_TOP_TRACKS_ENDPOINT = os.getenv('USER_TOP_TRACKS_ENDPOINT')
# genius api secrets
GENIUS_API_BASE_URL = os.getenv('GENIUS_API_BASE_URL')
GENIUS_API_ACCESS_TOKEN = os.getenv('GENIUS_API_ACCESS_TOKEN')
# instantiate genius client
genius = lg.Genius(GENIUS_API_ACCESS_TOKEN)

In [3]:
# Spotify API call helper functions.

def get_top_tracks():
    # TODO: this will be removed eventually as user-specific api calls will be made from the ui
    return requests.get(f"{SPOTIFY_API_BASE_URL}{USER_TOP_TRACKS_ENDPOINT}", headers = {
        "Authorization": f"Bearer {spotify.get_access_token()}"
    }).json()

def fetch_track_audio_features(track_id):
    return requests.get(f"{SPOTIFY_API_BASE_URL}{TRACK_AUDIO_FEATURES_ENDPOINT}{track_id}", headers = {
        "Authorization": f"Bearer {spotify.get_access_token()}"
    }).json()

def get_tracks(tracks):
    return requests.get(
        f"{SPOTIFY_API_BASE_URL}{TRACKS_ENDPOINT}",
        params = { "ids": ','.join(tracks) },
        headers = { "Authorization": f"Bearer {spotify.get_access_token()}" }
    ).json()

def get_artists(artist_ids):
    return requests.get(
        f"{SPOTIFY_API_BASE_URL}{ARTISTS_ENDPOINT}",
        params = { "ids": ','.join(artist_ids) },
        headers = { "Authorization": f"Bearer {spotify.get_access_token()}" }
    ).json()

In [4]:
def format_basic_artists_info(artists_json):

    artists = get_artists([artist['id'] for artist in artists_json])

    return [{
        'id': artist['id'],
        'name': artist['name'],
        'href': artist['href'],
        'followers': artist['followers']['total'] if 'followers' in artist else None,
        'genres': artist['genres'] if 'genres' in artist else None,
        'image': artist['images'][0] if 'images' in artist else None,
        'popularity': artist['popularity'] if 'popularity' in artist else None
    } for artist in artists["artists"]]

In [5]:
# Grabs basic track metadata, such as title and artist(s).
def grab_basic_track_info(track):
    artists = format_basic_artists_info(track['artists'])

    total_seconds = math.floor(track['duration_ms'] / 1000)
    leftover_seconds = total_seconds % 60
    duration = f"{total_seconds // 60}:{'0{}'.format(leftover_seconds) if leftover_seconds < 10 else leftover_seconds}"

    return [track['id'], track['name'], artists, duration,
            track['album']['id'], track['album']['name'], track['album']['images'][0]]

In [6]:
# Grabs the track's audio features from Spotify.
def get_track_audio_features(track_id):
    audio_features = fetch_track_audio_features(track_id)
    return [audio_features['danceability'], audio_features['energy'], audio_features['key'],
              audio_features['loudness'], audio_features['mode'], audio_features['speechiness'],
              audio_features['acousticness'], audio_features['instrumentalness'], audio_features['liveness'],
              audio_features['valence'], audio_features['tempo']]

In [7]:
# Grabs the track's lyrics from Genius. If no song lyrics exist, insert None into lyrics col
def get_track_lyrics(name, artist):
  try:
    song = genius.search_song(name, artist)
    return song.lyrics
  except AttributeError:
    return None

In [11]:
"""
Handler function to construct the track dataset.
Delegates information gathering and cleaning to three different helper functions, each of which is resonsible for a different data category.
"""
def construct_track_dataset(track_ids):
    complete_tracks = get_tracks(track_ids)

    return [
        grab_basic_track_info(track) +
        get_track_audio_features(track['id']) +
        [ get_track_lyrics(track['name'], ', '.join([artist['name'] for artist in track['artists']])) ]
        for track in complete_tracks['tracks']
    ]

In [12]:
# TODO: will be able to remove this upon spinning up ui
get_track_ids = lambda tracks_json: [t['id'] for t in tracks_json['items']]

In [13]:
top_tracks_json = get_top_tracks()
tracks = construct_track_dataset(get_track_ids(top_tracks_json))

Searching for "Decatur" by 6LACK...
Done.
Searching for "Georgia Peach" by Quinn XCII...
Done.
Searching for "Lady Writer" by Dire Straits...
Done.
Searching for "KANTE (feat. Fave)" by Davido, Fave...
Done.
Searching for "Assumptions - Kaytranada Edit" by Sam Gellaitry, KAYTRANADA...
No results found for: 'Assumptions - Kaytranada Edit Sam Gellaitry, KAYTRANADA'
Searching for "Believe" by NORRA, lostinspace...
No results found for: 'Believe NORRA, lostinspace'
Searching for "Sweet Virginia" by The Rolling Stones...
Done.
Searching for "Coastal Cat" by Tez Cadey...
No results found for: 'Coastal Cat Tez Cadey'
Searching for "Try Me" by The Weeknd...
Done.
Searching for "High Noon" by Kota the Friend, Statik Selektah...
Done.
Searching for "Bad Boy Boogie" by AC/DC...
Done.
Searching for "Running In Circles" by Mat Kearney...
Done.
Searching for "Shine On You Crazy Diamond (Pts. 1-5)" by Pink Floyd...
Done.
Searching for "Jersey Giant" by Elle King...
Done.
Searching for "4EVA (feat. Ph

In [14]:
tracks_df = pd.DataFrame(tracks, columns=[
    'Track ID', 'Track Name', 'Artists', 'Duration', 'Album ID', 'Album Name', 'Image',
    'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness',
    'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Lyrics'
])
display(tracks_df)

,Track ID,Track Name,Artists,Duration,Album ID,Album Name,Image,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Lyrics
0,5SMUyfT2P1kpKqWbkAResU,Decatur,"[{'id': '4IVAbR2w4JJNJDDRFP3E83', 'name': '6LA...",2:57,39JZhYotu5zFYJ8jw1D8bF,Since I Have A Lover,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.506,0.676,1,-6.168,1,0.3310,0.38700,0.285000,0.3830,0.4280,195.843,"12 ContributorsDecatur Lyrics[Intro]\nDa-da, l..."
1,0TfMitxzrMd2NTLxNXL2Hu,Georgia Peach,"[{'id': '3ApUX1o6oSz321MMECyIYd', 'name': 'Qui...",2:21,68ht4TaMromCg26Fu3fBB0,The People's Champ (Extended Version),"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.774,0.698,11,-5.448,0,0.0479,0.01130,0.000065,0.0676,0.8670,97.002,6 ContributorsGeorgia Peach Lyrics[Chorus]\nSh...
2,3mwFncaI2HBczQ92GP9MQF,Lady Writer,"[{'id': '0WwSkZ7LtFUFjGjMZBMt6T', 'name': 'Dir...",3:49,6HEOoO8aHq5M88OA4teVY1,Communiqué,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.633,0.798,1,-9.313,0,0.0281,0.16600,0.617000,0.0812,0.9630,147.645,16 ContributorsLady Writer Lyrics[Verse 1]\nLa...
3,7vKXc90NT5WBm3UTT4iTVG,KANTE (feat. Fave),"[{'id': '0Y3agQaa6g2r0YmHPOO9rh', 'name': 'Dav...",3:14,6lI21W76LD0S3vC55GrfSS,Timeless,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.724,0.809,6,-5.022,0,0.0929,0.18200,0.000000,0.0765,0.6060,99.005,8 ContributorsKANTE Lyrics[Intro]\nDamie\nYou'...
4,7lWdyj9RrMsEN6F0Dl0irE,Assumptions - Kaytranada Edit,"[{'id': '07UJz804RJxqNvxFXC3h9H', 'name': 'Sam...",4:48,5YTiQmOzu1m293UgtWNJSR,Assumptions (Kaytranada Edit),"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.722,0.780,5,-8.623,0,0.0753,0.00659,0.047000,0.0545,0.2000,119.997,None
5,18atM21acJCu9bH0YCsYG0,Believe,"[{'id': '41TOmzyp8cryPs7PXHKdb7', 'name': 'NOR...",2:13,3VLYzHg6S1JngbXYapwMF3,Believe,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.648,0.679,10,-6.240,1,0.1510,0.06450,0.012300,0.1000,0.1100,119.935,None
6,1hJrWWK74fKL7eeV3CFOvF,Sweet Virginia,"[{'id': '22bE4uQ6baNwSHPVcDxLCe', 'name': 'The...",4:26,5U4dnRZsfW8NmwBBkELFPh,Exile On Main Street (2010 Re-Mastered),"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.477,0.740,9,-5.397,1,0.0446,0.50200,0.000001,0.2630,0.5600,107.384,19 ContributorsSweet Virginia Lyrics[Verse 1]\...
7,7aFfZ8sqPgMDxBdoV9mclI,Coastal Cat,"[{'id': '5cBeFQv3kBVP8o15CmPTKb', 'name': 'Tez...",4:52,6PEGEok2ASsFO0yJMhsUZ2,Coastal Cat,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.763,0.381,9,-12.992,0,0.0746,0.02550,0.837000,0.0837,0.1790,118.994,None
8,4ppTAJUbNXELZcoUaL90wo,Try Me,"[{'id': '1Xyo4u8uXC1ZmMpatF05PJ', 'name': 'The...",3:41,4qZBW3f2Q8y0k1A84d4iAO,"My Dear Melancholy,","{'height': 640, 'url': 'https://i.scdn.co/imag...",0.456,0.679,8,-5.778,0,0.0640,0.59600,0.000573,0.1290,0.1070,92.026,113 ContributorsTranslationsTürkçeEspañolРусск...
9,03gTp2zMOTL36xDsfzgJlh,High Noon,"[{'id': '2AfU5LYBVCiCtuCCfM7uVX', 'name': 'Kot...",3:04,5rJ8SJ38ask8SN4ftwxvWS,To See A Sunset,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.524,0.799,8,-7.250,1,0.2770,0.06430,0.000000,0.2430,0.7570,82.090,6 ContributorsHigh Noon Lyrics[Intro]\nOne for...


In [15]:
for track in tracks:
    print(json.dumps(track[2], indent=2))

[
  {
    "id": "4IVAbR2w4JJNJDDRFP3E83",
    "name": "6LACK",
    "href": "https://api.spotify.com/v1/artists/4IVAbR2w4JJNJDDRFP3E83",
    "followers": 4346099,
    "genres": [
      "atl hip hop",
      "melodic rap",
      "r&b",
      "rap",
      "trap"
    ],
    "image": {
      "height": 640,
      "url": "https://i.scdn.co/image/ab6761610000e5ebf8e7a2d1a01fd98e43ee57dc",
      "width": 640
    },
    "popularity": 72
  }
]
[
  {
    "id": "3ApUX1o6oSz321MMECyIYd",
    "name": "Quinn XCII",
    "href": "https://api.spotify.com/v1/artists/3ApUX1o6oSz321MMECyIYd",
    "followers": 765288,
    "genres": [
      "indie pop rap"
    ],
    "image": {
      "height": 640,
      "url": "https://i.scdn.co/image/ab6761610000e5eb695a1945a0c36d5a70d570a2",
      "width": 640
    },
    "popularity": 65
  }
]
[
  {
    "id": "0WwSkZ7LtFUFjGjMZBMt6T",
    "name": "Dire Straits",
    "href": "https://api.spotify.com/v1/artists/0WwSkZ7LtFUFjGjMZBMt6T",
    "followers": 6820218,
    "genres": 